In [31]:
import dash
from dash import dcc, html, Input, Output
import dash_leaflet as dl
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import logging

In [32]:
# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# --- Database Configuration ---
DB_CONFIG = {
    "dbname": "lombardia_air_quality",
    "user": "airdata_user", 
    "password": "user",
    "host": "localhost",
    "port": "5432"
}

In [33]:
# --- Fetch data from PostgreSQL ---
def fetch_data():
    """Fetch station data from PostgreSQL database with error handling"""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        query = """
        SELECT nomestazione, lat, lng, nometiposensore
        FROM station
        WHERE lat IS NOT NULL AND lng IS NOT NULL;
        """
        df = pd.read_sql(query, conn)
        conn.close()
        logger.info(f"Fetched {len(df)} records from database")
        return df
    except Exception as e:
        logger.error(f"Database connection failed: {e}")
        # Return sample data for testing
        return None

# --- Create Dash App ---
app = dash.Dash(__name__)
app.title = "Lombardia Air Quality Monitor"

In [34]:
# Initial data fetch
try:
    df_all = fetch_data()
    pollutants = sorted(df_all["nometiposensore"].unique())
except Exception as e:
    logger.error(f"Initial data fetch failed: {e}")
    df_all = pd.DataFrame()
    pollutants = []

def create_layer_group(df, pollutant):
    """Create a layer group with markers for a specific pollutant"""
    if df.empty:
        return dl.LayerGroup([], id=f"layer-{pollutant}")
    
    df_filtered = df[df["nometiposensore"] == pollutant]
    markers = []
    
    for _, row in df_filtered.iterrows():  # Fixed syntax error
        marker = dl.Marker(
            position=[row["lat"], row["lng"]],  # Use list format
            children=dl.Popup(
                html.Div([
                    html.H4(row['nomestazione']),
                    html.P(f"Pollutant: {pollutant}"),
                    html.P(f"Coordinates: {row['lat']:.4f}, {row['lng']:.4f}")
                ])
            )
        )
        markers.append(marker)
    
    return dl.LayerGroup(markers, id=f"layer-{pollutant}")

/var/folders/r_/cwng275d0_lc1wkltzfdkw280000gn/T/ipykernel_1952/2770456592.py:11: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

INFO:__main__:Fetched 984 records from database


In [35]:
# --- Enhanced Layout ---
app.layout = html.Div([
    # Header
    html.Div([
        html.H1("🌍 Air Quality Monitoring - Lombardia", 
                style={
                    "textAlign": "center",
                    "margin": "20px",
                    "fontFamily": "Arial, sans-serif",
                    "fontWeight": "bold",
                    "color": "#2c3e50",
                    "textShadow": "1px 1px 2px rgba(0,0,0,0.1)"
                }),
        html.P("Real-time monitoring of air quality station across Lombardia region",
               style={
                   "textAlign": "center",
                   "fontFamily": "Arial, sans-serif",
                   "color": "#7f8c8d",
                   "marginBottom": "20px"
               })
    ]),
    
    # Map container
    html.Div([
        # Main map
        dl.Map(
            id="map",
            center=[45.5, 9.2],  # Centered on Lombardia
            zoom=8,
            children=[
                dl.TileLayer(
                    url="https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png",
                    attribution='&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors'
                )
            ],
            style={
                "width": "100%", 
                "height": "80vh", 
                "position": "relative",
                "border": "2px solid #bdc3c7",
                "borderRadius": "8px"
            }
        ),
        
        # Control panel
        html.Div([
            html.Label("📊 Select Pollutant:", 
                      style={"fontWeight": "bold", "marginBottom": "8px", "display": "block"}),
            dcc.Dropdown(
                id="pollutant-selector",
                options=[{"label": pol, "value": pol} for pol in pollutants] if pollutants else [],
                value=pollutants[0] if pollutants else None,
                clearable=False,
                placeholder="Select a pollutant..." if not pollutants else None
            ),
            html.Hr(style={"margin": "15px 0"}),
            html.Div(id="station-count", style={"fontSize": "14px", "color": "#7f8c8d"})
        ],
        style={
            "position": "absolute",
            "top": "20px",
            "right": "20px",
            "zIndex": "1000",
            "width": "280px",
            "backgroundColor": "rgba(255, 255, 255, 0.95)",
            "padding": "20px",
            "border": "1px solid #bdc3c7",
            "borderRadius": "8px",
            "boxShadow": "0 4px 12px rgba(0,0,0,0.15)",
            "fontFamily": "Arial, sans-serif",
            "backdropFilter": "blur(5px)"
        })
    ], style={"position": "relative", "margin": "20px"}),
    
    # Footer
    html.Div([
        html.P("Data source: Lombardia Environmental Agency | Last updated: Real-time",
               style={
                   "textAlign": "center",
                   "fontSize": "12px",
                   "color": "#95a5a6",
                   "margin": "20px",
                   "fontFamily": "Arial, sans-serif"
               })
    ])
])

In [ ]:
# --- Callbacks ---
@app.callback(
    [Output("map", "children"),
     Output("station-count", "children")],
    [Input("pollutant-selector", "value")]
)
def update_map_layers(selected_pollutant):
    """Update map layers based on selected pollutant"""
    try:
        # Fetch fresh data
        df_live = fetch_data()
        
        # Base tile layer
        base_layer = dl.TileLayer(
            url="https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png",
            attribution='&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors'
        )
        
        layers = [base_layer]
        station_info = "No pollutant selected"
        
        if selected_pollutant and not df_live.empty:
            # Add pollutant layer
            pollutant_layer = create_layer_group(df_live, selected_pollutant)
            layers.append(pollutant_layer)
            
            # Count station for this pollutant
            station_count = len(df_live[df_live["nometiposensore"] == selected_pollutant])
            station_info = f"📍 {station_count} station monitoring {selected_pollutant}"
        
        return layers, station_info
        
    except Exception as e:
        logger.error(f"Error updating map: {e}")
        base_layer = dl.TileLayer(
            url="https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png",
            attribution='&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors'
        )
        return [base_layer], "⚠️ Error loading data"

# --- Run Server ---
if __name__ == "__main__":
    print("🚀 Starting Lombardia Air Quality Dashboard...")
    print("📱 Open your browser to: http://127.0.0.1:8050")
    print("🔄 Press Ctrl+C to stop the server")
    
    app.run(
        debug=True,
        host='127.0.0.1',
        port=8050,
        dev_tools_ui=True,
        dev_tools_props_check=True
    )

🚀 Starting Lombardia Air Quality Dashboard...
📱 Open your browser to: http://127.0.0.1:8050
🔄 Press Ctrl+C to stop the server


/var/folders/r_/cwng275d0_lc1wkltzfdkw280000gn/T/ipykernel_1952/2770456592.py:11: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

INFO:__main__:Fetched 984 records from database
INFO:__main__:Fetched 984 records from database
INFO:__main__:Fetched 984 records from database
INFO:__main__:Fetched 984 records from database
